In [3]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmeans1').getOrCreate()


In [4]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
df = spark.read.load("./finalEdited-HumanHappiness-BDAS-mining.csv", format="csv", header=True, inferSchema=True)

# vector Assembler

In [6]:
# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
assembler = VectorAssembler(
    inputCols=['Year',
 'Rule of Law',
 'Disappearances, Conflicts, and Terrorism',
 'Women Security & Safety',
 'Security & Safety',
 'Women Movement',
 'Movement',
 'Legal and Regulatory Restrictions',
 'State Control over Internet Access',
 'Expression & Information',
 'Same Sex Relationships',
 'Divorce',
 'PERSONAL FREEDOM (Score)',
 'PERSONAL FREEDOM (Rank)',
 'Government  enterprises and investment',
 'Top marginal income tax rate',
 'Legal enforcement of contracts',
 'Reliability of police',
 'Gender Legal Rights Adjustment',
 'Money growth',
 'Inflation: Most recent year',
 'Compliance costs of importing and exporting',
 'Regulatory trade barriers',
 'Foreign ownership/investment restrictions',
 'Freedom to trade internationally',
 'Hiring regulations and minimum wage',
 'Labour market regulations',
 'Licensing restrictions',
 'Business regulations',
 'ECONOMIC FREEDOM (Score)',
 'ECONOMIC FREEDOM (Rank)'],
    outputCol="features")

In [7]:
# Now that we've created the assembler variable, let's actually transform the data.
output = assembler.transform(df)

In [8]:
output.show(2, truncate=False)

+----+---------+--------------------------+---------------------+--------------------+------------------------+-----------+----------------------------------------+-----------------------+-----------------+--------------+-----------+---------------------------------+----------------------------------+------------------------+----------------------+-------+------------------------+-----------------------+--------------------------------------+----------------------------+------------------------------+---------------------+------------------------------+------------+---------------------------+-------------------------------------------+-------------------------+-----------------------------------------+--------------------------------+-----------------------------------+-------------------------+----------------------+--------------------+------------------------+-----------------------+----------------------------------------------------------------------------------------------------

# StandardScaler

In [9]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
standard_scaler = StandardScaler(inputCol="features", outputCol="scaled")

In [10]:
train = standard_scaler.fit(output).transform(output)

In [11]:
train.select("scaled").show(2, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [12]:
scalerModel = standard_scaler.fit(output)

In [13]:
# Normalize each feature to have unit standard deviation.
cluster_final_data = scalerModel.transform(output)

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('find_hacker').getOrCreate()
from pyspark.ml.clustering import KMeans

In [15]:
kmeans3 = KMeans(featuresCol='scaled',k=3)
kmeans2 = KMeans(featuresCol='scaled',k=2)
kmeans4 = KMeans(featuresCol='scaled',k=4)
kmeans5 = KMeans(featuresCol='scaled',k=5)

In [16]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)
model_k4 = kmeans4.fit(cluster_final_data)
model_k5 = kmeans5.fit(cluster_final_data)

In [21]:
wssse_k3 = model_k3.computeCost(cluster_final_data)
wssse_k2 = model_k2.computeCost(cluster_final_data)
wssse_k4 = model_k4.computeCost(cluster_final_data)
wssse_k5 = model_k5.computeCost(cluster_final_data)

In [22]:
print("With K=3")
print("Within Set Sum of Squared Errors = " + str(wssse_k3))
print('--'*30)
print("With K=2")
print("Within Set Sum of Squared Errors = " + str(wssse_k2))
print('--'*30)
print("With K=4")
print("Within Set Sum of Squared Errors = " + str(wssse_k4))
print('--'*30)
print("With K=4")
print("Within Set Sum of Squared Errors = " + str(wssse_k5))

With K=3
Within Set Sum of Squared Errors = 30557.633703834457
------------------------------------------------------------
With K=2
Within Set Sum of Squared Errors = 37108.50877482409
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 27455.056435769737
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 25050.72873867154


In [24]:
for k in range(2,10):
    kmeans = KMeans(featuresCol='scaled',k=k)
    model = kmeans.fit(cluster_final_data)
    wssse = model.computeCost(cluster_final_data)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
Within Set Sum of Squared Errors = 37108.50877482409
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 30557.633703834457
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 27455.056435769737
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 25050.72873867154
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 24183.74249648865
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 23061.537249005807
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 22285.251136750332
------------------------------------------------------------
With K=9
Within Set Sum of Squared Errors = 21794.84046553681
------------------------------------------------------------


In [25]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  618|
|         2|  913|
|         0|   38|
+----------+-----+



In [26]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  903|
|         0|  666|
+----------+-----+



In [27]:
model_k4.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  507|
|         3|   32|
|         2|  378|
|         0|  652|
+----------+-----+



In [28]:
model_k5.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  468|
|         3|  597|
|         4|  279|
|         2|  193|
|         0|   32|
+----------+-----+



In [32]:
# Training a k-means model for cluster=3... we will do below steps for which this model fits best.
kmeans3 = KMeans(featuresCol='scaled',k=3)
model = kmeans3.fit(cluster_final_data)

In [36]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[7.04673899e+02 3.41151629e-01 6.76496378e-01 5.27056692e-01
 1.10320872e+00 1.92559263e-01 9.66160408e-02 6.58504255e-01
 7.83352898e-02 6.95904594e-01 9.03937798e-02 1.52031098e-01
 7.78774139e-01 6.48407540e-01 1.97353049e-01 5.44746681e-01
 2.27610718e-01 1.83387108e-01 6.29721008e+00 6.91774371e+00
 3.38086328e+00 1.64206105e-01 2.50202777e-01 2.23219410e-01
 1.08282434e+00 3.15335008e-01 4.80721995e-01 0.00000000e+00
 6.63368639e-01 1.63568902e+00 6.71666358e-01]
[705.26023654   3.93923296   4.96941562   3.77530985   5.23925132
   2.6555125    3.40966101   3.46245767   2.1484776    5.04864155
   2.33306296   2.51910213   5.13596533   0.72291994   2.5141543
   2.54933071   2.97835431   2.68460077   7.14318958   8.59715283
   6.66763752   3.04577305   3.8579562    3.16666948   5.63641776
   2.17236478   4.00013809   3.72424722   4.73108354   6.39059145
   0.78232431]
[705.3666053    2.58110396   4.27139035   2.73421793   4.15455392
   1.84774872   2.42080172   3.3

In [38]:
model.transform(cluster_final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         1|
|         2|
|         2|
|         2|
|         1|
|         1|
|         1|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
+----------+
only showing top 20 rows

